In [1]:
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off

In [1]:
!nvidia-smi

Tue Oct 17 23:05:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:31:00.0 Off |                    0 |
| N/A   45C    P0             172W / 300W |  44555MiB / 81920MiB |     41%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [3]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [4]:
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [5]:
################################################################################
# transformers parameters
################################################################################

# The pre-trained model from the Hugging Face Hub to load and fine-tune
model_name = "meta-llama/Llama-2-13b-hf"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

In [6]:
# Load model from Hugging Face Hub with model name and bitsandbytes configuration

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(model_name, bnb_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/mdafifal.mamun/miniconda3/envs/triagerx/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
dataset_path = "openj9_issues_cleaned.csv"

In [9]:
# Load dataset
dataset = load_dataset("csv", data_files = dataset_path, split = "train")


In [10]:
split_dataset = dataset.train_test_split(test_size=0.05)

In [11]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [12]:
dataset[0]

{'Unnamed: 0': 0,
 'issue_number': 10,
 'issue_url': 'https://github.com/eclipse-openj9/openj9/pull/10',
 'issue_title': 'Update README.md with proper AdoptOpenJDK link',
 'issue_body': 'Also remove the the coming soon reference, since the OpenJ9 build is\r\nlive now.\r\n\r\nSigned-off-by: Peter Shipton <Peter_Shipton@ca.ibm.com>',
 'issue_state': 'closed',
 'creator': 'pshipton',
 'comments': None,
 'assignees': 'charliegracie'}

In [13]:
print(f'Number of training prompts: {len(train_dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of training prompts: 4710
Column names are: ['Unnamed: 0', 'issue_number', 'issue_url', 'issue_title', 'issue_body', 'issue_state', 'creator', 'comments', 'assignees']


In [14]:
developers_list = set(dataset["assignees"])
print(developers_list)

{'knn-k', '0xdaryl', 'fengxue-IS', 'llxia', 'ChengJin01', 'pshipton', 'charliegracie', 'keithc-ca', 'gacholio', 'amicic', 'jdmpapin', 'ymanton', 'mpirvu', 'babsingh', 'andrewcraik', 'fjeremic', 'hzongaro', 'DanHeidinga', 'joransiu', 'vijaysun-omr', 'tajila', 'a7ehuo', 'dmitripivkine', 'gita-omr', 'JasonFengJ9', 'dsouzai'}


In [15]:
instruction_template = "Recommend a developer to solve the issue from one of the 26 developers:\n\n" + "\n".join(developers_list)
print(instruction_template)

Recommend a developer to solve the issue from one of the 26 developers:

knn-k
0xdaryl
fengxue-IS
llxia
ChengJin01
pshipton
charliegracie
keithc-ca
gacholio
amicic
jdmpapin
ymanton
mpirvu
babsingh
andrewcraik
fjeremic
hzongaro
DanHeidinga
joransiu
vijaysun-omr
tajila
a7ehuo
dmitripivkine
gita-omr
JasonFengJ9
dsouzai


In [16]:
def _get_input_context(sample):
    return f"Issue Title: {sample['issue_title']}\n\nIssue Description: {sample['issue_body']}"


def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{instruction_template}"
    input_context = f"{INPUT_KEY}\n{_get_input_context(sample)}"
    response = f"{RESPONSE_KEY}\n{sample['assignees']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [17]:
def create_query_format(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{instruction_template}"
    input_context = f"{INPUT_KEY}\n{_get_input_context(sample)}"
    # response = f"{RESPONSE_KEY}\n{sample['assignees']}"
    # end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [18]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [19]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [20]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = dataset.column_names,
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [21]:
# Random seed
seed = 33

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, train_dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/4710 [00:00<?, ? examples/s]

Map:   0%|          | 0/4710 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4710 [00:00<?, ? examples/s]

In [22]:
preprocessed_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4600
})

In [23]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [24]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [25]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [26]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

In [27]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

In [28]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 25

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [29]:
fine_tune(model,
      tokenizer,
      preprocessed_dataset,
      lora_r,
      lora_alpha,
      lora_dropout,
      bias,
      task_type,
      per_device_train_batch_size,
      gradient_accumulation_steps,
      warmup_steps,
      max_steps,
      learning_rate,
      fp16,
      logging_steps,
      output_dir,
      optim)

LoRA module names: ['down_proj', 'v_proj', 'k_proj', 'o_proj', 'up_proj', 'gate_proj', 'q_proj']
All Parameters: 6,734,566,400 || Trainable Parameters: 62,586,880 || Trainable Parameters %: 0.9293379303528732
Training...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.285800
2,3.010800
3,2.934300
4,2.684500
5,2.218000
6,1.751500
7,1.521600
8,1.336200
9,0.994800
10,0.706900


***** train metrics *****
  epoch                    =       0.02
  total_flos               =  1662756GF
  train_loss               =     1.2779
  train_runtime            = 0:01:34.91
  train_samples_per_second =      1.054
  train_steps_per_second   =      0.263
{'train_runtime': 94.9193, 'train_samples_per_second': 1.054, 'train_steps_per_second': 0.263, 'total_flos': 1785371494502400.0, 'train_loss': 1.2778915667533874, 'epoch': 0.02}
Saving last checkpoint of the model...


In [31]:
# Load fine-tuned weights
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = device, torch_dtype = torch.bfloat16)
# Merge the LoRA layers with the base model
model = model.merge_and_unload()

# Save fine-tuned model at a new location
output_merged_dir = "results/openj9_llama2_13b"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

# Save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

('results/openj9_llama2_13b/tokenizer_config.json',
 'results/openj9_llama2_13b/special_tokens_map.json',
 'results/openj9_llama2_13b/tokenizer.model',
 'results/openj9_llama2_13b/added_tokens.json',
 'results/openj9_llama2_13b/tokenizer.json')

In [32]:
from transformers import StoppingCriteria, StoppingCriteriaList

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
stop_list = ['\nHuman:', '\n```\n', "### End"]

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [33]:
import re

def inference(index, test_ds):
    pipe = pipeline(task="text-generation", model=model, max_length=max_length, tokenizer=tokenizer, stopping_criteria=stopping_criteria)
    result = pipe(create_query_format(test_ds[index])["text"])

    response_text = result[0]["generated_text"]
    pattern = r"### Response:\n.*?\n"

    response = re.findall(pattern, response_text, re.DOTALL | re.MULTILINE | re.IGNORECASE)
    
    return {
        "LLM Response": response,
        "Actual": test_dataset[index]["assignees"]
    }

In [34]:
import random

for i in range (10):
    try:
        data_index = random.randint(0, len(test_dataset) - 1)
        print(f"Index: {data_index}")
        out = inference(data_index, test_dataset)
        print(out, "\n\n")
    except:
        pass

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Index: 163
{'LLM Response': ['### Response:\nDanHeidinga\n'], 'Actual': 'DanHeidinga'} 


Index: 28
{'LLM Response': ['### Response:\nDanHeidinga\n'], 'Actual': 'keithc-ca'} 


Index: 6
{'LLM Response': ['### Response:\nmpirvu\n'], 'Actual': 'DanHeidinga'} 


Index: 189
{'LLM Response': ['### Response:\nDanHeidinga\n'], 'Actual': 'keithc-ca'} 


Index: 70
{'LLM Response': ['### Response:\npshipton\n'], 'Actual': 'keithc-ca'} 


Index: 62
{'LLM Response': ['### Response:\nmpirvu\n'], 'Actual': 'a7ehuo'} 


Index: 57
{'LLM Response': ['### Response:\npshipton\n'], 'Actual': '0xdaryl'} 


Index: 35
{'LLM Response': ['### Response:\npshipton\n'], 'Actual': 'mpirvu'} 


Index: 188
{'LLM Response': ['### Response:\nDanHeidinga\n'], 'Actual': 'pshipton'} 


Index: 26
{'LLM Response': ['### Response:\npshipton\n'], 'Actual': 'pshipton'} 


